In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import tensorflow as tf
from helper_functions import *
import pandas as pd
importTensorflow(memory=4090)

2.15.0
1 Physical GPUs, 1 Logical GPUs


In [2]:
import tensorflow_hub as hub
model_load = tf.keras.models.load_model("79_USE", custom_objects={'KerasLayer': hub.KerasLayer})
model_load.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense_2 (Dense)             (None, 101)               51813     
                                                                 
Total params: 256849637 (979.80 MB)
Trainable params: 256849637 (979.80 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [3]:
df = pd.read_csv("queries.csv")
df.head()

,String,Class
0,Rich and buttery delicacy recipes,foie_gras
1,Luxurious appetizer with duck liver,foie_gras
2,Indulgent gourmet dish preparation,foie_gras
3,Recipes for savory liver pâté,foie_gras
4,Delicate and decadent French cuisine ideas,foie_gras


In [4]:
d = set()
counter = -1
l = []
for thing in df.Class:
    if thing in d:
      l.append(counter)
    else:
        counter += 1
        d.add(thing)
        l.append(counter)

In [5]:
class_list = []
for x in df.Class:
    if x not in class_list:
        class_list.append(x)

In [6]:
class_list[:5]

['foie_gras', 'club_sandwich', 'cheese_plate', 'cup_cakes', 'garlic_bread']

In [7]:
df['classEncode'] = l

In [8]:
df = df.sample(frac = 1)
df.head()

,String,Class,classEncode
1855,Sauce blends for enhancing the flavor of choco...,chocolate_cake,74
2180,Simple recipes for appetizers featuring homema...,deviled_eggs,88
149,Delicious dairy-based dessert for hot days,ice_cream,6
144,Seasoning blends for enhancing the flavor of p...,gnocchi,5
258,Simple recipes for gourmet dinners featuring b...,filet_mignon,10


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.String, df.classEncode, test_size=0.1, random_state=69)

In [10]:
one_hot_encoded_train = tf.one_hot(y_train, depth=101)
one_hot_encoded_test = tf.one_hot(y_test, depth=101)

In [11]:
train_data_one_hot = tf.data.Dataset.from_tensor_slices((X_train, one_hot_encoded_train)).shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)
test_data_one_hot = tf.data.Dataset.from_tensor_slices((X_test, one_hot_encoded_test)).batch(64).prefetch(tf.data.AUTOTUNE)

In [12]:
model_load.evaluate(test_data_one_hot)

4/4 [==============================] - 2s 8ms/step - loss: 0.2411 - accuracy: 0.9597


[0.2410961389541626, 0.9596773982048035]

In [13]:
class_list[tf.argmax(model_load.predict(['crisp and sweet']), axis=1).numpy()[0]]

1/1 [==============================] - 0s 338ms/step


'foie_gras'